In [4]:
pip install webdriver-manager


  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [56]:
from selenium import webdriver #used for automating interactions with web pages
from webdriver_manager.chrome import ChromeDriverManager #manages ChromeDriver setup
from selenium.webdriver.chrome.service import Service #allows configuration for running the ChromeDriver service
from bs4 import BeautifulSoup #used for parsing HTML code and extracting data
import pandas as pd #used for working with structured data and dataframes


In [58]:
service = Service(ChromeDriverManager().install()) #automatically installs and starts the ChromeDriver service

In [61]:
driver = webdriver.Chrome(service=service)#passes the ChromeDriver service to the driver to link it with the browser


In [17]:
based_url = 'https://dom.kz/prodazha/kvartiry/astana?pageNum='
links = []
#loop through all pages (1-60)
for page in range(1, 61):
    url = based_url + str(page)
    driver.get(url) #opens the current page using the web driver
    soup = BeautifulSoup(driver.page_source, 'html.parser') #parses the HTML source of the page
    cards = soup.find_all('div', class_='lister-content-list container mt-4') #finds all listing cards on the page
    for card in cards: 
        card_links = card.find_all('a', href=True) #finds all links within the listing cards
        for link in card_links:
            href = str(link.get('href')) #extracts the href attribute of the link
            if href.__contains__('listing'): #checks if the link contains the word "listing"
                links.append(href) #appends the link to the links list
print(links) #prints all collected links to the listings

['/listing/385283', '/listing/386473', '/listing/385093', '/listing/384073', '/listing/378682', '/listing/386735', '/listing/386734', '/listing/386733', '/listing/386732', '/listing/386731', '/listing/386730', '/listing/386729', '/listing/386728', '/listing/386727', '/listing/386726', '/listing/386725', '/listing/386724', '/listing/386723', '/listing/386721', '/listing/386720', '/listing/386719', '/listing/386718', '/listing/386717', '/listing/386711', '/listing/386710', '/listing/386706', '/listing/386704', '/listing/386702', '/listing/386701', '/listing/386700', '/listing/386699', '/listing/386698', '/listing/386697', '/listing/386696', '/listing/386695', '/listing/386694', '/listing/386693', '/listing/386692', '/listing/386691', '/listing/386690', '/listing/386689', '/listing/386688', '/listing/386687', '/listing/386686', '/listing/386685', '/listing/386684', '/listing/386683', '/listing/386682', '/listing/386681', '/listing/386680', '/listing/386471', '/listing/386470', '/listing/3

In [18]:
len(links)

900

In [19]:
url='https://dom.kz'
url_ = url + '/listing/378682' #forming the URL for a specific listing
driver.get(url_) #opening the listing page in the browser using the driver
soup = BeautifulSoup(driver.page_source, 'html.parser') #loading the html of the page for parsing
information = soup.find_all('div', class_='housing-shower container') #finding the main container with listing information
#iterating through the found information
for info in information:
    name = info.find('div', attrs={'class': 'housing-shower__title housing-shower__header--title col-12 col-lg-8'}).text
    name = name.split(',') #splitting the title into parts to extract specific data
    first = name[0].strip().split(' ') 
    room = first[1].strip() #extracting the number of rooms
    residence = first[6:] #extracting the name of the residential complex
    residence = ' '.join(residence) #removing unnecessary spaces from the name
    m2 = name[1].strip() #extracting the apartment area
    city  = name[2].strip() #extracting the city
    district = ' '
    if len(name) == 4: #checking if the district exists
        district = name[3].strip()
    address = " "
    if len(name) == 5:  #checking if the address exists
        address = name[4].strip()
    price = info.find('div', attrs={'class': 'main-info-title-mobile__price'}).text 
    price_parts = price.split('₸')[:2]
    total_price = price_parts[0].strip()
    price_m2 = price_parts[1].strip()
#searching for additional property characteristics
    year = info.find('div', attrs={'class': 'housing-shower-properties-list'})
    year_spl = year.find_all('div', attrs={'class':'housing-shower-properties-list__item'})
    year_build = ' ' #year of construction
    expl = ' '  #completion status
    class_build = ' ' #housing class
    type_build = ' ' #type/material of walls
    for i  in year_spl: 
        t = i.text
        if t.__contains__('Год'): #finding the year of construction
            t = t.split(' ')
            year_build = t[3]
        elif t.__contains__('Сдан/не'): #finding completion status
            t = t.split(' ')
            expl = t[4:]
            expl = ' '.join(expl)
        elif t.__contains__('Класс жилья'): #finding housing class
            t = t.split(' ')
            class_build = t[2]
        elif t.__contains__('Тип/материал'): #finding type/material of walls
            t = t.split(' ')
            type_build = t[3]
#printing extracted data for verification          
    print(room) 
    print(residence.strip()) #outputs the residential complex name
    print(city)
    print(district)
    print(address)
    print(m2)
    print(total_price)
    print(price_m2)
    print(year_build)
    print(expl)
    print(class_build)
    print(type_build)

2-комнатной
GreenLine.Flora
г. Астана
 
ул. Е 900 4
50.1 м²
35 500 000
708 583
2024
Не сдан
Бизнес
Монолитные


In [20]:
apartment_table = pd.DataFrame(columns=['Rooms', 'Residential complex', 'City', 'District', 'Address', 'Apartment area', 'Price', 'Price/m2', 'Year', 'Commissioned/not commissioned', 'Housing class',  'Material'])
apartment_table

,Rooms,Residential complex,City,District,Address,Apartment area,Price,Price/m2,Year,Commissioned/not commissioned,Housing class,Material


In [21]:
URL='https://dom.kz'
for link in links: #iterate through each listing link
    URL_ = URL + link #construct the full URL for the listing
    driver.get(URL_) #load the listing page using WebDriver
    soup = BeautifulSoup(driver.page_source, 'html.parser') #parse the html content of the page
    information = soup.find_all('div', class_='housing-shower container') #find the main container with listing details
    for info in information:  #process the information for the listing
        name = info.find('div', attrs={'class': 'housing-shower__title housing-shower__header--title col-12 col-lg-8'}).text
        name = name.split(',') #split the title into parts
        first = name[0].strip().split(' ') 
        room = first[1].strip() #extract the number of rooms
        residence = first[6:] #extract the name of the residential complex
        residence = ' '.join(residence) 
        m2 = name[1].strip() #extract the apartment area
        city  = name[2].strip() #extract the city
        district = ' ' 
        if len(name) == 4: #if the district is mentioned
            district = name[3].strip() 
        address = " "
        if len(name) == 5: #if the address is mentioned
            address = name[4].strip()
        price = info.find('div', attrs={'class': 'main-info-title-mobile__price'}).text
        price_parts = price.split('₸')[:2]
        total_price = price_parts[0].strip() #extract the total price
        price_m2 = price_parts[1].strip() #extract the price per square meter
        year = info.find('div', attrs={'class': 'housing-shower-properties-list'})
        year_spl = year.find_all('div', attrs={'class':'housing-shower-properties-list__item'})
        year_build = ' ' #year of construction
        expl = ' '  #completion status
        class_build = ' ' #housing class
        type_build = ' ' #wall type/material
        for i  in year_spl: #process additional characteristics
            t = i.text
            if t.__contains__('Год'):
                t = t.split(' ')
                year_build = t[3]
            elif t.__contains__('Сдан/не'):
                t = t.split(' ')
                expl = t[4:]
                expl = ' '.join(expl)
            elif t.__contains__('Класс жилья'):
                t = t.split(' ')
                class_build = t[2]
            elif t.__contains__('Тип/материал'):
                t = t.split(' ')
                type_build = t[3]
    row = [room, residence.strip(), city, district, address,  m2, total_price, price_m2, year_build, expl, class_build, type_build]
    apartment_table.loc[len(apartment_table)] = row #add the row to the dataframe

In [22]:
apartment_table

,Rooms,Residential complex,City,District,Address,Apartment area,Price,Price/m2,Year,Commissioned/not commissioned,Housing class,Material
0,4-комнатной,Anar,г. Астана,,ул. Орынбор 13,127 м²,75 000 000,590 551,2023,Сдан,Комфорт,Кирпичные
1,2-комнатной,Garden View,г. Астана,,ул. Бухар жырау 26 стр,47.68 м²,43 950 000,921 770,2024,Не сдан,Бизнес+,Монолитные
2,3-комнатной,Самрук Towers,г. Астана,,ул. Кайыма Мухамедханова 17,86 м²,38 000 000,441 860,2018,Сдан,Эконом,Монолитные
3,1-комнатной,GreenLine.Headliner,г. Астана,,ул. Толе би 50,45 м²,37 500 000,833 333,2022,Не сдан,Бизнес,
4,2-комнатной,GreenLine.Flora,г. Астана,,ул. Е 900 4,50.1 м²,35 500 000,708 583,2024,Не сдан,Бизнес,Монолитные
...,...,...,...,...,...,...,...,...,...,...,...,...
895,2-комнатной,AQJUREK,г. Астана,ул. Улы дала,,62.96 м²,27 576 480,438 000,2025,Не сдан,Эконом,
896,2-комнатной,AQJUREK,г. Астана,ул. Улы дала,,62.96 м²,27 576 480,438 000,2025,Не сдан,Эконом,
897,2-комнатной,AQJUREK,г. Астана,ул. Улы дала,,59.43 м²,29 358 420,494 000,2025,Не сдан,Эконом,
898,1-комнатной,AQJUREK,г. Астана,ул. Улы дала,,34.38 м²,18 633 960,542 000,2025,Не сдан,Эконом,


In [23]:
apartment_table.to_csv('apartment.csv')